In [8]:
import torch
import torch.nn as nn
from torch import nn, optim
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import copy
import pickle
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

In [9]:
# Load the data
file_path = "./static_data_summary.csv"
data = pd.read_csv(file_path)

# Extract the 'totpop' column
totpop = data['totpop'].values

# Extract the real travel demand data from ridesouring file 
ridesourcing_file_path = "./Ridesourcing_CensusCount_ALL_0_Filled.csv"
ridesourcing_data = pd.read_csv(ridesourcing_file_path)

# Aggregate travel demand data by summing all time intervals for each census tract
travel_demand = ridesourcing_data.drop(columns=['index']).sum(axis=1).values

# Convert 'totpop' to a tensor and move it to the appropriate device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
populations = torch.tensor(totpop, dtype=torch.float32).to(device)
travel_demand_tensor = torch.tensor(travel_demand, dtype=torch.float32).to(device)

In [10]:
# Extract features from the data
features = data[['SerHourBusRoutes', 'SerHourRailRoutes', 'BusStopDen', 'RailStationDen', 'popden', 'pctmale', 
                 'pctbachelor', 'young2', 'pcthisp', 'carown', 'pctlowinc', 'pctmidinc', 'pcthighinc', 
                 'pctsinfam2', 'CrimeDen', 'RdNetwkDen', 'InterstDen']].values

# Normalize the data
# Min-max normalization for features, populations, and travel demand 
features_min = np.min(features, axis=0)
features_max = np.max(features, axis=0)
features_normalized = (features - features_min) / (features_max - features_min)

pop_min = np.min(totpop)
pop_max = np.max(totpop)
populations_normalized = (totpop - pop_min) / (pop_max - pop_min)

travel_demand_min = np.min(travel_demand)
travel_demand_max = np.max(travel_demand)
travel_demand_normalized = (travel_demand - travel_demand_min) / (travel_demand_max - travel_demand_min)

In [11]:
# Convert to tensors
X_train = torch.tensor(features_normalized, dtype=torch.float32).to(device)
y_train = torch.tensor(travel_demand_normalized, dtype=torch.float32).to(device).view(-1, 1)
populations = torch.tensor(populations_normalized, dtype=torch.float32).to(device)

In [12]:
# Creating the model: Neural network with 1 hidden layer  
class SimpleNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

In [13]:
# Computes a monotonicity penalty if the predictions do not uphold monotonic constraint 
def calculate_monotonicity_penalty(preds, populations, device):
    penalty = torch.zeros(1, device=device)
    for i in range(1, len(preds)):
        if populations[i] > populations[i - 1] and preds[i] < preds[i - 1]:
            penalty += torch.abs(preds[i] - preds[i - 1])
    return penalty

# Loss function: combines the MSE loss with the monotonicity penalty 
def calculate_loss(y_pred, label, criterion, lam, device, populations):
    cost1 = criterion(y_pred, label).double()
    monotonicity_penalty = calculate_monotonicity_penalty(y_pred, populations, device)
    cost = (1 - lam) * cost1 + lam * monotonicity_penalty # lam controls the trade-off between MSE loss and monotonicity penalty 
    return cost, cost1, monotonicity_penalty

In [14]:
# Define model, criterion, and optimizer
input_size = X_train.shape[1]
hidden_size = 50
output_size = 1
model = SimpleNN(input_size, hidden_size, output_size).to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
lam = 0.5  # Lambda for the monotonicity penalty

# Training loop
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    
    # Forward pass
    y_pred = model(X_train)
    
    # Calculate loss
    loss, cost1, monotonicity_penalty = calculate_loss(
        y_pred, y_train, criterion, lam, device, populations
    )
    
    # Backward pass and optimization
    loss.backward()
    optimizer.step()
    
    # Print loss for monitoring
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}, Cost1: {cost1.item()}, Monotonicity Penalty: {monotonicity_penalty.item()}')

print("Training completed")

Epoch [1/100], Loss: 2.5521223545074463, Cost1: 0.011854110285639763, Monotonicity Penalty: 5.092390537261963
Epoch [2/100], Loss: 2.403026580810547, Cost1: 0.011245746165513992, Monotonicity Penalty: 4.794807434082031
Epoch [3/100], Loss: 2.2611329555511475, Cost1: 0.010689293965697289, Monotonicity Penalty: 4.5115766525268555
Epoch [4/100], Loss: 2.125251054763794, Cost1: 0.010191548615694046, Monotonicity Penalty: 4.2403106689453125
Epoch [5/100], Loss: 1.9948266744613647, Cost1: 0.009746846742928028, Monotonicity Penalty: 3.9799065589904785
Epoch [6/100], Loss: 1.8702645301818848, Cost1: 0.00934935174882412, Monotonicity Penalty: 3.731179714202881
Epoch [7/100], Loss: 1.753743290901184, Cost1: 0.008995863609015942, Monotonicity Penalty: 3.498490810394287
Epoch [8/100], Loss: 1.6444907188415527, Cost1: 0.008681044913828373, Monotonicity Penalty: 3.2803003787994385
Epoch [9/100], Loss: 1.5396002531051636, Cost1: 0.008396994322538376, Monotonicity Penalty: 3.07080340385437
Epoch [10/1